In [9]:
%pip install bs4


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 13.5 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pandas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

url = "https://bio.iiserb.ac.in/faculty"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
html = response.text

print(html[:500])

<style>

    
.MultiCarousel {
	float: left;
	overflow: hidden;
	padding: 15px;
	width: 100%;
	position: relative;
}
.MultiCarousel .MultiCarousel-inner {
	transition: 1s ease all;
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item {
	float: left;
}

/* .item{
    width: 200px;
} */

.MultiCarousel .MultiCarousel-inner .item > div {
	text-align: center;
	padding: 10px;
	margin: 10px;
	/* background: #EFF0F5;
	color: #E4E4E4; */
}
.MultiCarousel .leftLst,
.Mul


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bio.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [24]:
import requests

url = "https://bio.iiserb.ac.in/faculty"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
html = response.text

print(html[:500])

<style>

    
.MultiCarousel {
	float: left;
	overflow: hidden;
	padding: 15px;
	width: 100%;
	position: relative;
}
.MultiCarousel .MultiCarousel-inner {
	transition: 1s ease all;
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item {
	float: left;
}

/* .item{
    width: 200px;
} */

.MultiCarousel .MultiCarousel-inner .item > div {
	text-align: center;
	padding: 10px;
	margin: 10px;
	/* background: #EFF0F5;
	color: #E4E4E4; */
}
.MultiCarousel .leftLst,
.Mul


In [23]:
from bs4 import BeautifulSoup
import requests
import csv
import time  # optional, to pause between requests

BASE_URL = "https://bio.iiserb.ac.in"

# 1️⃣ Load main page
url = f"{BASE_URL}/Home/Faculty"
page = requests.get(url)
soup = BeautifulSoup(html, "html.parser")

# 2️⃣ Select all faculty blocks
faculty_blocks = soup.select("div#myTabContent div.col-md-3.prdct-grid")

# 3️⃣ Open CSV
with open("faculty_bhopbio.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Name", "Profile Link", "Designation", "Email",
        "Research Area", "Room", "Phone", "Image Link",
        "PhD", "Postdoc"
    ])

    for block in faculty_blocks:
        # main page info
        img_tag = block.select_one("img")
        img_link = img_tag.get("src").strip() if img_tag else ""

        name_tag = block.select_one("div.product-name strong")
        name = name_tag.text.strip() if name_tag else ""
        profile_tag = block.select_one("div.product-name a[href^='https']")
        profile_link = profile_tag.get("href").strip() if profile_tag else ""

        p_tag = block.select_one("div.product-name p")
        text_lines = [t.strip() for t in p_tag.get_text(separator="\n").split("\n") if t.strip()] if p_tag else []

        designation = text_lines[0] if len(text_lines) > 0 else ""
        email = next((t for t in text_lines if "@" in t), "")
        research = next((t for t in text_lines if " and " in t or "Biology" in t or "Chemistry" in t), "")
        room = next((t for t in text_lines if "Room" in t), "")
        phone = next((t for t in text_lines if "+91" in t or t.replace(" ", "").isdigit()), "")

        phd = ""
        postdoc = ""

        if profile_link:
            try:
                 profile_page = requests.get(profile_link)
                 profile_soup = BeautifulSoup(profile_page.text, "html.parser")
   
            # The "Academic Details" section is inside div with id="collapseOne-m"
                 academic_section = profile_soup.find("div", id="collapseOne-m")

                 if academic_section:
            # Each academic detail is within <div class="row list"><utility-title>...</utility-title></div>
                    for tag in academic_section.select("div.row.list utility-title"):
                       text = tag.get_text(strip=True)

                       if "Ph.D" in text or "PhD" in text:
                          phd = text
                       elif "Postdoctoral" in text or "Postdoc" in text:
                          postdoc = text

            except Exception as e:
               print(f"Failed to fetch {profile_link}: {e}")
        # 5️⃣ Write row to CSV
        writer.writerow([
            name, profile_link, designation, email, research, room, phone, img_link, phd, postdoc
        ])
